In [8]:
!pip install geopandas
!pip install rasterio
!pip install selenium
!pip install pycrs

In [9]:
!pip install lxml

In [1]:
#-*- coding:utf-8 -*-
import os, zipfile
import geopandas
import rasterio
from rasterio.plot import show
from rasterio.plot import show_hist
from rasterio.mask import mask
from shapely.geometry import Point
from pyproj import Proj, transform
import math
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver import ActionChains
import time
from lxml import etree
import glob
from shapely.geometry import box
from fiona.crs import from_epsg
import pycrs
from pyproj import Geod
geod = Geod(ellps='GRS80')

#+proj=longlat +ellps=GRS80 +datum=NAD83 +no_defs 

cmd1 = '!pip install selenium'

driver_cnt = 0

wildfire = '/home/qweddww123/farsite/TrainDataGen/WildfireShapefiles'
ffdriverpath = '/home/qweddww123/farsite/TrainDataGen/GetLCPdata'
initialdownloadfilepath = '/home/qweddww123/Downloads'
extractfilepath = '/home/qweddww123/farsite/TrainDataGen/GetLCPdata/LCP/lcp_extract'

fp = webdriver.FirefoxProfile('/home/qweddww123/.mozilla/firefox/0lnpvwfz.default-release')
fp.set_preference('browser.download.folderList', 2)
fp.set_preference('browser.download.dir', initialdownloadfilepath)
fp.set_preference('browser.helperApps.neverAsk.OpenFile', "application/octet-stream, multipart/x-zip, application/x-zip-compressed, application/zip, application/gzip, application/zip-compressed, text/csv,application/x-msexcel,application/excel,application/x-excel,application/vnd.ms-excel,image/png,image/jpeg,text/html,text/plain,application/msword,application/xml")
fp.set_preference("browser.helperApps.neverAsk.SaveToDisk","application/octet-stream, multipart/x-zip, application/x-zip-compressed, application/zip, application/gzip, application/zip-compressed, text/csv,application/x-msexcel,application/excel,application/x-excel,application/vnd.ms-excel,image/png,image/jpeg,text/html,text/plain,application/msword,application/xml")
driver = webdriver.Firefox(firefox_profile=fp, executable_path=ffdriverpath+'/geckodriver') #firefox driver 다운로드 받고, 다운로드 받은 파일 경로 지정

coords_list1 = []

lat_dis_per_1km = 0.009
lon_dis_per_1km = 0.01

for year in sorted(os.listdir(wildfire), key=int):
    print(year, '***************************************************************************************************************')
    if not os.path.isdir(extractfilepath+'/'+year):
        os.system('mkdir '+extractfilepath+'/'+year)
    for target_fire in sorted(os.listdir(wildfire+'/'+year), key=str):
        path = wildfire+'/'+year
        if(target_fire[0] == '.'):
            continue
        elif(target_fire.split('_')[0] == 'ak' or target_fire.split('_')[0] == 'AK'):
            continue
        elif(target_fire.split('_')[0] == 'hi'):
            continue
        else:
            print()
            print('target_fire = ', target_fire)
            for wildfire_time in  sorted(os.listdir(path+'/'+target_fire), key=str):
                if(wildfire_time[0] != '.'):
                    #print('fire path = ', path+'/'+target_fire+'/'+wildfire_time)
                    for wildfire_shp in sorted(os.listdir(path+'/'+target_fire+'/'+wildfire_time), key=str):
                        extractedfile_targetpath = extractfilepath+'/'+year+'/'+target_fire
                        if not os.path.isdir(extractedfile_targetpath):
                            fileextension = os.path.splitext(wildfire_shp)[1]
                            if fileextension == '.shp':
                                if os.path.splitext(wildfire_shp)[0] == wildfire_time:
                                    print(os.path.splitext(wildfire_shp)[0])
                                    #print('missing fire = ', extractedfile_targetpath)
                                    driver_cnt += 1
                                    filepath = path+'/'+target_fire+'/'+wildfire_time+'/'+wildfire_shp
                                    #read shpfile
                                    shpfile = geopandas.read_file(filepath)
                                    print("filepath = ", filepath)

                                    #shpfile min, max coords
                                    try:
                                        x1 = float(shpfile.bounds.minx[0])
                                        y1 = float(shpfile.bounds.miny[0])
                                        x2 = float(shpfile.bounds.maxx[0])
                                        y2 = float(shpfile.bounds.maxy[0])
                                    except:
                                        break
                                    #print(y2,y1,x2,x1)

                                    #initial lon, lat: x1, y1
                                    az12, az21, dist = geod.inv(x1, y1, x2, y2)
                                    npts = 1
                                    del_s = dist/(npts+1)
                                    center = geod.fwd_intermediate(x1, y1, az12, npts=npts, del_s=del_s)
                                    print("center point = ", center.lons[0], center.lats[0])

                                    az12, az21, dist = geod.inv(x1, y1, center.lons[0], center.lats[0])
                                    print("azimuth from min point to the center = ", az12)

                                    new_x2, new_y2, max_backaz= geod.fwd(center.lons[0], center.lats[0], float(45.0), 21000*math.sqrt(2)/2.0)
                                    new_x1, new_y1, min_backaz= geod.fwd(center.lons[0], center.lats[0], float(-135.0), 21000*math.sqrt(2)/2.0)
                                    print('max coords = ', new_x2, ", ", new_y2)
                                    print('min coords = ', new_x1, ", ", new_y1)
                                    north = round(new_y2,8)
                                    south = round(new_y1,8)
                                    east = round(new_x2,8)
                                    west = round(new_x1,8)
                                    #print('north = ', north)
                                    #print('south = ', south)
                                    #print('east = ', east)
                                    #print('west = ', west)

                                    coords = str(north)+','+str(south)+','+str(east)+','+str(west)
                                    url = 'https://landfire.cr.usgs.gov/Website/distreq/RequestSummary.jsp?AL='+coords+'&PL=FIG61HZ,'
                                    print("TEST url")
                                    print(url)

                                    az12, az21, dist1 = geod.inv(center.lons[0], center.lats[0], new_x1, new_y1)
                                    #print("from center to min coords")
                                    #print(":", round(az12, 0), round(az21, 0))

                                    az12, az21, dist1 = geod.inv(new_x1, new_y1, new_x2, new_y2)
                                    #print("from new min to new max")
                                    #print(":", round(az12, 0), round(az21, 0))

                                    #print(round(dist1, 3))
                                    #dist = 21000*math.sqrt(2)
                                    #print(round(dist, 3))

                                    az12, az21, new_xdist = geod.inv(new_x1, new_y1, new_x2, new_y1)
                                    print('new_xdist = ', round(new_xdist/1000, 0))
                                    az12, az21, new_ydist = geod.inv(new_x1, new_y1, new_x1, new_y2)
                                    print('new_ydist = ', round(new_ydist/1000, 0))

                                    az12, az21, xdist = geod.inv(x1, y1, x2, y1)
                                    az12, az21, ydist = geod.inv(x1, y1, x1, y2)
                                    print("wildfire xdist = ", xdist/1000)
                                    print("wildfire ydist = ", ydist/1000)

                                    if xdist/1000 <= 10.0 and ydist/1000 <= 10.0:
                                        print('shapefile size < 100.0km^2')
                                        coords = str(north)+','+str(south)+','+str(east)+','+str(west)
                                        print('bounding coords = ', coords)

                                        #url의 위경도 수정 부분 코드 작성 필요
                                        #shp파일 읽어오고, 21km*21km에 해당하는 영역을 잡아내고 그걸 위,경도로 또 변환해서 아래 주소의 'AL=' 뒤에 입력
                                        #us_140 LCP
                                        #url = 'https://landfire.cr.usgs.gov/Website/distreq/RequestSummary.jsp?AL='+coords+'&PL=LC137HZ,'
                                        #us_200 LCP
                                        #url = 'https://landfire.cr.usgs.gov/Website/distreq/RequestSummary.jsp?AL='+coords+'&PL=FE954HZ,'
                                        url = 'https://landfire.cr.usgs.gov/Website/distreq/RequestSummary.jsp?AL='+coords+'&PL=FIG61HZ,'
                                        print(url)

                                        if driver_cnt == 1:
                                            driver.get(url)
                                            try:
                                                wait(driver, 3).until(EC.element_to_be_clickable((By.XPATH, '//form')))
                                                driver.find_element_by_xpath("//form").click()
                                            except:
                                                time.sleep(10)
                                                driver_cnt = 0
                                                break

                                            coords_list1.append(coords)
                                            coords_list1.append(target_fire)
                                            coords_list1.append(wildfire_time)
                                        else:
                                            driver.current_window_handle
                                            #open a new tab
                                            #execute_url = 'window.open('+'"'+url+'"'+');'
                                            #driver.execute_script(execute_url)
                                            #driver.switch_to.window(driver.window_handles[driver_cnt-1])
                                            driver.get(url)
                                            #time.sleep(1)
                                            try:
                                                wait(driver, 30).until(EC.element_to_be_clickable((By.XPATH, '//form')))
                                                driver.find_element_by_xpath("//form").click()
                                            except:
                                                time.sleep(10)
                                                print('time.sleep(10)')
                                                driver_cnt = 0
                                                break

                                            coords_list1.append(coords)
                                            coords_list1.append(target_fire)
                                            coords_list1.append(wildfire_time)

                                        print('driver_cnt = ', driver_cnt)

                                        if driver_cnt == 1:
                                            #delay for downloading lcp files
                                            time.sleep(180)
                                            #close tabs
                                            for i in range(len(driver.window_handles)):
                                                driver.switch_to.window(driver.window_handles[0])
                                                driver.close()
                                            fp = webdriver.FirefoxProfile('/home/qweddww123/.mozilla/firefox/0lnpvwfz.default-release')
                                            fp.set_preference('browser.download.folderList', 2)
                                            fp.set_preference('browser.download.dir', initialdownloadfilepath)
                                            fp.set_preference('browser.helperApps.neverAsk.OpenFile', "application/octet-stream, multipart/x-zip, application/x-zip-compressed, application/zip, application/gzip, application/zip-compressed, text/csv,application/x-msexcel,application/excel,application/x-excel,application/vnd.ms-excel,image/png,image/jpeg,text/html,text/plain,application/msword,application/xml")
                                            fp.set_preference("browser.helperApps.neverAsk.SaveToDisk","application/octet-stream, multipart/x-zip, application/x-zip-compressed, application/zip, application/gzip, application/zip-compressed, text/csv,application/x-msexcel,application/excel,application/x-excel,application/vnd.ms-excel,image/png,image/jpeg,text/html,text/plain,application/msword,application/xml")
                                            driver = webdriver.Firefox(firefox_profile=fp, executable_path=ffdriverpath+'/geckodriver') #firefox driver 다운로드 받고, 다운로드 받은 파일 경로 지정                                                                            

                                            list_of_files = glob.glob(initialdownloadfilepath+'/*') # * means all if need specific format then *.csv
                                            latest_file = max(list_of_files, key=os.path.getctime)

                                            #extract all .zip files on extract_file_path
                                            print('latest_file = ', latest_file)

                                            zipfilepath = latest_file
                                            with zipfile.ZipFile(zipfilepath, 'r') as zip_ref:
                                                zipfilename = latest_file.split('.')[0]
                                                zipfilename = zipfilename.split('/')[4]
                                                extractedwildfiredir = extractfilepath+'/'+zipfilename
                                                print("extractedwildfiredir = ", extractedwildfiredir)
                                                #os.system('touch '+wildfiredirname)
                                                zip_ref.extractall(extractedwildfiredir)
                                                time.sleep(5)
                                            #read xml files and extract the bounding box coordinates
                                            string_list = []
                                            for xmlfile in os.listdir(extractedwildfiredir):
                                                ext = os.path.splitext(xmlfile)[1]
                                                str1 = ""
                                                if ext == '.xml':
                                                    tree = etree.parse(extractedwildfiredir+'/'+xmlfile)
                                                    westbc = tree.xpath('//westbc/text()')
                                                    eastbc = tree.xpath('//eastbc/text()')
                                                    northbc = tree.xpath('//northbc/text()')
                                                    southbc = tree.xpath('//southbc/text()')
                                                    for ele in northbc:
                                                        str1 += ele
                                                    str1 += ','
                                                    for ele in southbc:
                                                        str1 += ele
                                                    str1 += ','
                                                    for ele in eastbc:
                                                        str1 += ele
                                                    str1 += ','
                                                    for ele in westbc:
                                                        str1 += ele
                                                    string_list.append(str1)

                                            print(string_list)
                                            print(coords_list1)
                                            #file name change
                                            cnt = 0

                                            for i in range(len(coords_list1)):
                                                if i%3 == 0:
                                                    for j in range(len(string_list)):
                                                        if coords_list1[i] == string_list[j]:
                                                            newdirname = coords_list1[3*int(i/3+1)-2]
                                                            print(newdirname)
                                                            newfilename = newdirname
                                                            newextractedwildfiredir = extractfilepath+'/'+year+'/'+newdirname
                                                            os.system('mv '+extractedwildfiredir+' '+newextractedwildfiredir)                                                        
                                                            time.sleep(20)
                                                        
                                    else:
                                        print('shapefile size is larger than 100.0km^2')
                                
                                    driver_cnt = 0
                                
                                    #clear list
                                    try:
                                        os.system('rm -r '+extractfilepath+'/lf*')
                                        os.system('rm -r '+extractedwildfiredir+'home')
                                    except:
                                        continue
                                    time.sleep(5)                                    
                                    coords_list1.clear()                                    
                                    print()
                break

for i in range(len(driver.window_handles)):
    driver.switch_to.window(driver.window_handles[0])
    driver.close()
    
print("downloading lcp files completed!!")

2016 ***************************************************************************************************************

target_fire =  al_caney_head

target_fire =  al_half_way

target_fire =  al_lake_chinnabee

target_fire =  al_lookout_mountain

target_fire =  al_power_horn

target_fire =  ar_pipeline

target_fire =  ar_sweetwater

target_fire =  ar_whitaker_point

target_fire =  az_airstrip

target_fire =  az_baca

target_fire =  az_baker_canyon

target_fire =  az_baldwin

target_fire =  az_bert

target_fire =  az_black_peak

target_fire =  az_brown

target_fire =  az_burro

target_fire =  az_cedar

target_fire =  az_choulic

target_fire =  az_coco

target_fire =  az_cowboy

target_fire =  az_cumero

target_fire =  az_fill

target_fire =  az_freeze

target_fire =  az_fresnal

target_fire =  az_fuller

target_fire =  az_fulton

target_fire =  az_jack

target_fire =  az_juniper

target_fire =  az_la_sierra

target_fire =  az_lion_fta

target_fire =  az_maple

target_fire =  az_mormon

t

rm: cannot remove '/home/qweddww123/farsite/TrainDataGen/GetLCPdata/LCP/lcp_extract/lf*': No such file or directory
rm: cannot remove '/home/qweddww123/farsite/TrainDataGen/GetLCPdata/LCP/lcp_extract/lf*': No such file or directory
rm: cannot remove '/home/qweddww123/farsite/TrainDataGen/GetLCPdata/LCP/lcp_extract/lf*': No such file or directory
rm: cannot remove '/home/qweddww123/farsite/TrainDataGen/GetLCPdata/LCP/lcp_extract/lf*': No such file or directory
rm: cannot remove '/home/qweddww123/farsite/TrainDataGen/GetLCPdata/LCP/lcp_extract/lf*': No such file or directory



target_fire =  fl_taylor

target_fire =  ga_brow_fire

target_fire =  ga_burrell

target_fire =  ga_burrell_42

target_fire =  ga_chimney_top

target_fire =  ga_creek_road

target_fire =  ga_fox_mountain_fire

target_fire =  ga_irwin_mill

target_fire =  ga_rock_mountain

target_fire =  ga_rocky_face

target_fire =  ga_rough_ridge

target_fire =  ga_tatum_gulf

target_fire =  ga_timber_ridge

target_fire =  ga_treat_mountain

target_fire =  id_black

target_fire =  id_boundary

target_fire =  id_buck

target_fire =  id_cedar

target_fire =  id_comet

target_fire =  id_dry_creek

target_fire =  id_elk_ridge

target_fire =  id_fleck

target_fire =  id_freeman

target_fire =  id_gleason

target_fire =  id_granite

target_fire =  id_grape

target_fire =  id_gun_club

target_fire =  id_harlan_creek

target_fire =  id_henrys_creek
20160823_0000
filepath =  /home/qweddww123/farsite/TrainDataGen/WildfireShapefiles/2016/id_henrys_creek/20160823_0000/20160823_0000.shp
center point =  -111.81070

rm: cannot remove '/home/qweddww123/farsite/TrainDataGen/GetLCPdata/LCP/lcp_extract/lf*': No such file or directory
rm: cannot remove '/home/qweddww123/farsite/TrainDataGen/GetLCPdata/LCP/lcp_extract/lf*': No such file or directory
rm: cannot remove '/home/qweddww123/farsite/TrainDataGen/GetLCPdata/LCP/lcp_extract/lf*': No such file or directory
rm: cannot remove '/home/qweddww123/farsite/TrainDataGen/GetLCPdata/LCP/lcp_extract/lf*': No such file or directory
rm: cannot remove '/home/qweddww123/farsite/TrainDataGen/GetLCPdata/LCP/lcp_extract/lf*': No such file or directory
rm: cannot remove '/home/qweddww123/farsite/TrainDataGen/GetLCPdata/LCP/lcp_extract/lf*': No such file or directory
rm: cannot remove '/home/qweddww123/farsite/TrainDataGen/GetLCPdata/LCP/lcp_extract/lf*': No such file or directory
rm: cannot remove '/home/qweddww123/farsite/TrainDataGen/GetLCPdata/LCP/lcp_extract/lf*': No such file or directory
rm: cannot remove '/home/qweddww123/farsite/TrainDataGen/GetLCPdata/LCP/


target_fire =  wv_south_fire

target_fire =  wy_arden

target_fire =  wy_berry

target_fire =  wy_broadway

target_fire =  wy_buffalo

target_fire =  wy_carrot

target_fire =  wy_cedar_draw_2

target_fire =  wy_cliff_creek

target_fire =  wy_cottonwood_creek

target_fire =  wy_deer_creek

target_fire =  wy_fawn

target_fire =  wy_hatchery

target_fire =  wy_hunter_peak
2016
filepath =  /home/qweddww123/farsite/TrainDataGen/WildfireShapefiles/2016/wy_hunter_peak/2016/2016.shp
center point =  -109.6968707821723 44.86693414580967
azimuth from min point to the center =  60.23077591343353
max coords =  -109.56379066315536 ,  44.96134054707056
min coords =  -109.82951620476793 ,  44.772371607919425
TEST url
https://landfire.cr.usgs.gov/Website/distreq/RequestSummary.jsp?AL=44.96134055,44.77237161,-109.56379066,-109.8295162&PL=FIG61HZ,
new_xdist =  21.0
new_ydist =  21.0
wildfire xdist =  6.631101842964765
wildfire ydist =  3.7872795538593396
shapefile size < 100.0km^2
bounding coords =  44.

...



target_fire =  wa_kusshi_creek

target_fire =  wa_left_hand

target_fire =  wa_mt._dana

target_fire =  wa_north_mill_creek

target_fire =  wa_south_fork_creek

target_fire =  wa_williams_flats
20190803_2233
filepath =  /home/qweddww123/farsite/TrainDataGen/WildfireShapefiles/2019/wa_williams_flats/20190803_2233/20190803_2233.shp
center point =  -118.56217635257944 47.96115102580817
azimuth from min point to the center =  59.882197649863016
max coords =  -118.42132235210377 ,  48.05549731646786
min coords =  -118.70251788120687 ,  47.86663107138448
TEST url
https://landfire.cr.usgs.gov/Website/distreq/RequestSummary.jsp?AL=48.05549732,47.86663107,-118.42132235,-118.70251788&PL=FIG61HZ,
new_xdist =  21.0
new_ydist =  21.0
wildfire xdist =  11.173476100617847
wildfire ydist =  6.463592247603713
shapefile size is larger than 100.0km^2


rm: cannot remove '/home/qweddww123/farsite/TrainDataGen/GetLCPdata/LCP/lcp_extract/lf*': No such file or directory
rm: cannot remove '/home/qweddww123/farsite/TrainDataGen/GetLCPdata/LCP/lcp_extract/lf50133553_us_210lcp13_21home': No such file or directory




target_fire =  wy_bomber_lake

target_fire =  wy_brimstone

target_fire =  wy_fishhawk
2019
filepath =  /home/qweddww123/farsite/TrainDataGen/WildfireShapefiles/2019/wy_fishhawk/2019/2019.shp
center point =  -109.86092867057775 44.37734191300942
azimuth from min point to the center =  44.762522447204454
max coords =  -109.7289659819337 ,  44.47175775318994
min coords =  -109.99246759361934 ,  44.28277253830226
TEST url
https://landfire.cr.usgs.gov/Website/distreq/RequestSummary.jsp?AL=44.47175775,44.28277254,-109.72896598,-109.99246759&PL=FIG61HZ,
new_xdist =  21.0
new_ydist =  21.0
wildfire xdist =  5.937324520496931
wildfire ydist =  5.978572506176275
shapefile size < 100.0km^2
bounding coords =  44.47175775,44.28277254,-109.72896598,-109.99246759
https://landfire.cr.usgs.gov/Website/distreq/RequestSummary.jsp?AL=44.47175775,44.28277254,-109.72896598,-109.99246759&PL=FIG61HZ,
driver_cnt =  1
latest_file =  /home/qweddww123/Downloads/lf79786759_us_210lcp13_21.zip
extractedwildfiredi

rm: cannot remove '/home/qweddww123/farsite/TrainDataGen/GetLCPdata/LCP/lcp_extract/lf*': No such file or directory
rm: cannot remove '/home/qweddww123/farsite/TrainDataGen/GetLCPdata/LCP/lcp_extract/lf79786759_us_210lcp13_21home': No such file or directory




target_fire =  wy_nowater

target_fire =  wy_pedro_mountain

target_fire =  wy_stink_water
downloading lcp files completed!!
